In [2]:
###### Giada Barzaghi - August 2021
### Modeling aviation networks: hybrid networks 

### Goal 1. integrating the concept of itinerary 
### Goal 2. insert some more realistic values for the variables in the model (eg. from Seymour, EMEP/EEA spreadsheet)
### Goal 3: REPRESENT YOUR NETWORK GRAPHICALLY + other graphs 

# SETTING UP WORKING ENVIRONMENT
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyomo.environ import *

In [3]:
# CREATING SETS AND ASSIGNING VALUES TO VARIABLES

# set of airports 
N = [n for n in range(1,10)]

In [4]:
# set of markets ---> note: remember difference city-pair / airport-pair
M = [(n1, n2) 
    for n1 in N 
    for n2 in N 
     if n1!=n2]
#print(M)

In [5]:
# set of flight-legs
F = M
#print(F)

In [6]:
# ---> DISREGARD FOR NOW
# set of itineraries (Seba's method) 
I1 = [(n1, n2, n3)
    for n1 in N
    for n2 in N
    for n3 in N
    if n1!=n2
    if n1!=n3
    if n3!=n2]
#print(I1)

In [7]:
# ---> DISREGARD FOR NOW
# here the last airport can be the same as the staring one 
I2 = []
for f in F:
    for m in M:
        if f[-1]== m[0]:
            c = (m[-1],)
            b = f+c
            I2.append(b)
        
#print(I2)

In [8]:
# One - stop itineraries
I3 = []
for f in F:
    for m in M:
        if f[-1]== m[0] and f[0]!=m[-1]:
            c = (m[-1],)
            b = f+c
            I3.append(b)
        
#print(I3)

In [9]:
# 2 - stop itineraries
I4 =[]
for i in I3:
    for m in M:
        if m[-1]==i[0] and m[0] not in i:
            c = list(i)
            c.pop(0)
            c = tuple(c)
            b = m+c
            I4.append(b)
#print(I4)

In [10]:
# Merging all itineraries (with 0, 1 or 2 stops) together to get the complete set of itineraries 
I = F+I3+I4
#print(I)
# len(I)

# Can merge all elements of a tuple together as one string to obtain tuples of same dimension
Y=[]
for i in I:
    i = str(i)
    Y.append(''.join(i))
    
#print(Y)

#for y in Y:
#    print(len(y))

# TO DO : CAN TAKE A RANDOM SAMPLE OUT OF THIS SET TO BUILD A SMALLER NETWORK + graphic representation 


In [11]:
# IMPORTING DATA FROM EEA/EMEP AIR POLLUTANT EMISSION INVENTORY GUIDEBOOK 2019
# (saved in my GitHub directory)
url = 'https://raw.githubusercontent.com/GiadaBarzaghi/MSc-Dissertation-/main/EEA%3AEMEP%20air%20pollutant%20emission%20inventory%202019.csv'
df = pd.read_csv(url,index_col=0)
df.head()

# Subsetting the imported dataframe
df = df[['Sector', 'Type', 'Technology', 'Pollutant', 'Value', 'Unit']]
# Dropping null values
df = df.dropna()

# Creating list of aircraft models 
A1 = df[['Technology']].to_numpy()
print('Number of aircraft models from EEA/EMEP: '+str(len(A1)))

# aletrnative
# IMPORTING DATA FROM SEYMOUR ET AL. 2020 - FEAT MODEL 
url1 = 'https://raw.githubusercontent.com/kwdseymour/FEAT/master/aircraft_type_designators.csv'
aircraft_type_designators = pd.read_csv(url1, index_col=0)
A = aircraft_type_designators[['ac_code_icao']].to_numpy()
A = [tuple(n) for n in A]
print('Number of aircraft models from Seymour et al.2020: '+str(len(A)))

Number of aircraft models from EEA/EMEP: 464
Number of aircraft models from Seymour et al.2020: 133


In [12]:
df.info()
aircraft_type_designators.info()

<class 'pandas.core.frame.DataFrame'>
Index: 464 entries, 1.A.3.a.i.(i) to 1.A.3.a.ii.(i)
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sector      464 non-null    object 
 1   Type        464 non-null    object 
 2   Technology  464 non-null    object 
 3   Pollutant   464 non-null    object 
 4   Value       464 non-null    float64
 5   Unit        464 non-null    object 
dtypes: float64(1), object(5)
memory usage: 25.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 0 to 132
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ac_code_iata  133 non-null    object
 1   ac_code_icao  133 non-null    object
 2   bada_model    133 non-null    object
 3   ac_name_oag   133 non-null    object
 4   ac_name_ps    97 non-null     object
dtypes: object(5)
memory usage: 6.2+ KB


In [13]:
# Assigning a random capacity to each aircraft model 
# Look up realistic values
np.random.seed(12345)
K = {}
for a in A:
    K[a]=np.random.randint(100,300)


In [14]:
# Assigning random demand to each itinerary
# -> from normal distribution ----> THEN WITH MULTINOMIAL LOGIT
np.random.seed(12345)
D={}
for i in Y:
    D[i]=np.random.normal(100, 10)/100


In [19]:
# Assigning values for pollution to each aircraft type when on a specific itinerary 

##### Assigning a random distance to each itinerary 
#### General rule -> non-stop:short distance, one-stop:medium distance, two-stop: long distance
# ----> then assign pollution based on aircraft and distance 

Distance = {}
for y in Y:
    if len(y)==6:
        Distance[y] = np.random.randint(1000, 2000)
    elif len(y)==9:
        Distance[y] = np.random.randint(2000, 3500)
    else:
        Distance[y]= np.random.randint(3501, 5000)
#print(Distance)

# create a dictionary with aircraft type and relative pollution 
# select aircraft types that are in Seymour from ICAO list 
# if a == a in the new list 
# pollution = distance * pollution 
    
# Creating a column for Aircraft Type Designators (ATD) in df where ATD 
# is indicated in the same format as in Seymour's paper
ATD = []
for i in df['Technology']:
    ATD.append(i[0:4])
#print(ATD)

# Creating common column ATD in the 2 dataframes imported previously (from Seymour and from ICAO)
df['ATD']=ATD
aircraft_type_designators['ATD'] = aircraft_type_designators['ac_code_icao']

# Merging the 2 dataframes to assign values for pollution to aircrafts in Seymour
df1 = pd.merge(df, aircraft_type_designators, on='ATD' )
df1 = df1[['ATD', 'Pollutant', 'Value', 'Unit']]
#df1.info()

# Now lets only consider values for CO2 to start with 
# !!!!!!!!!!!! TO DO : convert values of other pollutants into CO2 equivalent 
df_CO2 = df1[df1['Pollutant']=='CO2']

# PROBLEM TO SOLVE: LTO is not considering the distance -> integrate results from Seymour 
# to calculate emissions according to distance 

# Assigning values for pollution (kg of CO2/LTO) for each aircraft type to each itinerary
# PROBLEM PERSISTS: values for pollutions are all the same as we are not considering distance 
# temporary solution: multiply values for LTO for itinerary distance (haven't tried yet)


P = {}
for i in Y:
    P[i]=df_CO2[['ATD', 'Value']]
    
print(dict(list(P.items())[0:2])) # NOT CORRECT: VALUES ARE DUPLICATED --> check dataframe creation 

Distance_df = pd.DataFrame.from_dict(Distance, orient='index', columns = ['Distance'])
#Distance_df.head()


#P_df = pd.DataFrame.from_dict(P, orient='index')
#P_df.head()


{'(1, 2)':       ATD     Value
2    B763   5449.29
14   B763   5449.29
20   B762   4607.37
22   B762   4607.37
37   B772   7580.19
41   B772   7580.19
48   B773   8072.95
60   B773   8072.95
65   B752   4292.19
77   B752   4292.19
87   B753   4610.47
91   B753   4610.47
97   B77W   9736.15
105  B77W   9736.15
114  B788  10944.46
126  B788  10944.46
131  A320   2570.93
136  A320   2570.93
149  A332   6829.44
156  A332   6829.44
163  A333   6829.44
165  A333   6829.44
176  A343   6362.65
180  A343   6362.65
193  A306   5427.89
195  A306   5427.89
211  A310   4821.24
216  A310   4821.24
227  A319   2169.76
230  A319   2169.76
240  B738   2775.47
254  B738   2775.47
256  A346  10624.82
258  A346  10624.82
277  B744  10456.98
287  B744  10456.98
297  B737   2597.65
303  B737   2597.65
304  A345  10329.23
306  A345  10329.23, '(1, 3)':       ATD     Value
2    B763   5449.29
14   B763   5449.29
20   B762   4607.37
22   B762   4607.37
37   B772   7580.19
41   B772   7580.19
48   B773   8072.9

In [22]:
df1[0:20]

,ATD,Pollutant,Value,Unit
0,B763,SOx,1.45,kg/LTO
1,B763,TSP,0.16,kg/LTO
2,B763,CO2,5449.29,kg/LTO
3,B763,HC,7.56,kg/LTO
4,B763,CO,29.65,kg/LTO
5,B763,Fuel use for LTO,1729.93,kg/LTO
6,B763,H2O,2127.82,kg/LTO
7,B763,TSP,0.16,kg/LTO
8,B763,H2O,2127.82,kg/LTO
9,B763,SOx,1.45,kg/LTO


In [182]:
df_CO2.head()

,ATD,Pollutant,Value,Unit
2,B763,CO2,5449.29,kg/LTO
14,B763,CO2,5449.29,kg/LTO
20,B762,CO2,4607.37,kg/LTO
22,B762,CO2,4607.37,kg/LTO
37,B772,CO2,7580.19,kg/LTO


In [83]:
# Creating a dummy for whether a flight-leg can be executed or not with a specific aircraft model:
# (0.5 probability of being yes/1)
R = {}
for i in Y:
    for a in A:
        if np.random.randn()<=0.5:
            R[i, a]=1
        else:
            R[i, a]=0

In [84]:
# Assigning random values for max utilization of each aircraft type
# LOOK UP REALISTIC ONES 
W = {}
for a in A:
    W[a] = np.random.randint(10, 50)

In [85]:
# NOT WORKING !!!! SET I CONTAINS ELEMENTS OF DIFFERENT DIMENSIONS, HOW DO YOU SOLVE THIS?
# TEMPORARY SOLUTION: using Y instead of I (itineraries as strings rather than integers)

# SETTING UP THE CONCRETE MODEL 
# initializing parmaters and sets
m = ConcreteModel()
m.I = Set(initialize=Y) # with 'dimen='' can only specify one dimension 
m.A = Set(initialize=A)
m.K=Param(m.A, initialize=K)
m.W = Param(m.A, initialize = W)
m.D=Param(m.I, initialize=D)
m.P=Param(m.I, m.A, initialize=P)
m.R=Param(m.I, m.A, initialize=R)

# setting up decision variables
m.f=Var(m.I, m.A, domain=NonNegativeIntegers) # f = frequency of flights operated by aircraft type A on itinerary I 
m.z=Var(m.A, domain=NonNegativeIntegers) # z= number of aircrafts type A

# setting up model objective and constraints
# OBJECTIVE
def minimize_pollution(m):
    return sum(m.f[i,a]*m.P[i,a]
              for i in m.I for a in m.A)
m.objective=Objective(rule=minimize_pollution, sense=minimize)

# CONSTRAINTS 
def utilization_rule(m, a):
    return sum(m.f[i,a]
              for i in m.I)<=m.W[a]*m.z[a]
m.utilization= Constraint(m.A, rule=utilization_rule)

def demand_rule(m, i):
    return sum(m.f[i, a]*m.K[a]
              for a in m.A)>=m.D[i]
m.demand_rule = Constraint(m.I, rule = demand_rule)

def operational_rule(model, i):
    return sum(m.f[i,a] 
              for I in m.I)<=m.R[i,a]*m.f[i,a]
m.operational_rule = Constraint(m.I, rule=operational_rule)


In [191]:
# calling the solver -> using cbc

#%%time # what do we need this for?
opt = SolverFactory('cbc', executable=r'/usr/local/bin/cbc')
opt.options['mipgap']=0.20 
results = opt.solve(m, tee=False) # tee=True to see the output of the model

In [ ]:
# PRINTING SOME RESULTS
# 1. number of aircrafts for each model type
for a in m.A:
    print('Aircraft type: '+str(a))
    print('Number of aircrafts: '+str(m.z[a].value))

# 2. frequancy of flights on itinerary i operated with aircraft type a
for i in m.I:
    for a in m.A:
        print('Flight leg: %s; \naircraft type: %s; \nnumber of flights operated: %s\n' 
             % (i, a, m.f[i,a].value))